## Generate single instrument soundtrack

In [2]:
from music21 import instrument

# Instrument list
instruments = {"Piano": instrument.Piano(),
               "Harpsichord": instrument.Harpsichord(),
               "Glockenspiel": instrument.Glockenspiel(),
               "Xylophone": instrument.Xylophone(),
               "ElectricOrgan": instrument.ElectricOrgan(),
               "Harmonica": instrument.Harmonica(),
               "AcousticGuitar": instrument.AcousticGuitar(),
               "ElectricGuitar": instrument.ElectricGuitar(),
               "ElectricBass": instrument.ElectricBass(),
               "Violin": instrument.Violin(),
               "Viola": instrument.Viola(),
               "Violoncello": instrument.Violoncello(),
               "Harp": instrument.Harp(),
               "Timpani": instrument.Timpani(),
               "Trombone": instrument.Trombone(),
               "Tuba": instrument.Tuba(),
               "SopranoSaxophone": instrument.SopranoSaxophone(),
               "Flute": instrument.Flute(),
               "Koto": instrument.Koto(),
               "Soprano": instrument.Soprano()}

# Tmpo list
tmpo_fctr = {"original": 1.00,
             "x125": 1.25,
             "x150": 1.50,
             "x075": 0.75,
             "x050": 0.50}
# Key
keys = [-2, -1, 0, 1, 2]

# major conversions
majors = dict([("A-", 4), ("A", 3), ("B-", 2), ("B", 1), ("C", 0), ("D-", -1),
              ("D", -2), ("E-", -3), ("E", -4), ("F", -5), ("G-", 6), ("G", 5)])
minors = dict([("A-", 1), ("A", 0), ("B-", -1), ("B", -2), ("C", -3),
              ("D-", -4), ("D", -5), ("E-", 6), ("E", 5), ("F", 4), ("G-", 3), ("G", 2)])

## Synthesize MIDI to audio (mp3)

In [3]:
from midi2audio import FluidSynth


def midi_to_audio(input_midi, output_audio):
    '''
    Using the default sound font in 44100 Hz sample rate
    '''
    fs = FluidSynth()
    fs.midi_to_audio(input_midi, output_audio)


## Generate single instrument soundtrack

In [13]:
from music21 import converter
from tqdm import tqdm
from pathlib import Path


_TARGET_AUDIO_PATH = rf'audio'
_MIDI_PATH = rf'audio/tmp'
_AUDIO_PATH = rf'audio/instruments'
_MID = rf'.midi'
_MP3 = rf'.mp3'
_MIDI_SOLO_CHOIR = rf'HighwayToHell_solo_choir'

score = converter.parse(Path(_TARGET_AUDIO_PATH,f'{_MIDI_SOLO_CHOIR}{_MID}'))

# Create instrument
for inst_name, inst_obj in tqdm(instruments.items()):
    score.parts[0].insert(0, inst_obj)
    # With diferent tempo
    # scale (in this case stretch) the overall tempo by this factor
    for tmpo_name, tempo in tmpo_fctr.items():
        mod_score = score.scaleOffsets(tempo).scaleDurations(tempo)

        # With diferent key
        for halfstep in keys:
            _SOLO_MIDI = ''
            if (halfstep == 0) and tmpo_name == 'original':
                _SOLO_MIDI = rf'HighwayToHell_solo_{inst_name}_{tmpo_name}{_MID}'
                mod_score.write('midi', Path(_MIDI_PATH, _SOLO_MIDI))
            else:
                new_key_score = mod_score.transpose(halfstep)
                key = new_key_score.analyze('key')
                _SOLO_MIDI = rf'HighwayToHell_solo_{inst_name}_{tmpo_name}_{key.tonic}{key.mode}{_MID}'
                new_key_score.write('midi', Path(_MIDI_PATH, _SOLO_MIDI))

            midi_to_audio(Path(_MIDI_PATH, _SOLO_MIDI), Path(
                _AUDIO_PATH, f'{_SOLO_MIDI[:-5]}{_MP3}'))


  0%|          | 0/20 [02:42<?, ?it/s]


KeyboardInterrupt: 

In [148]:
_AUDIO_ROOT_PATH = rf'audio/'
_MID = rf'.midi'
_MIDI_SOLO_CHOIR = rf'HighwayToHell_solo_choir'

score = converter.parse(rf'{_AUDIO_ROOT_PATH}{_MIDI_SOLO_CHOIR}{_MID}')
key=score.analyze('key')
# Print the key and the mode (major or minor)
print(key.tonic, key.mode)

newscore = score.transpose(-2)
new_key = newscore.analyze('key')
print(new_key.tonic.name, new_key.mode)

D major
C major


In [11]:
from pathlib import Path
_TARGET_AUDIO_PATH = rf'audio/'
_MIDI_PATH=rf'audio/tmp'
_AUDIO_PATH=rf'audio/instruments'
_MID = rf'.midi'
_MP3 = rf'.mp3'
_MIDI_SOLO_CHOIR = rf'HighwayToHell_solo_choir'


p=Path(_TARGET_AUDIO_PATH,_MIDI_SOLO_CHOIR)
print((str(p)))

audio/HighwayToHell_solo_choir


## Load a trained neural network model

In [ ]:
import tensorflow as tf
from musicnn_keras.tagger import top_tags


musicnn = tf.keras.models.load_model(
    './musicnn_keras/keras_checkpoints/MSD_musicnn.h5')

In [ ]:
top_tags(f'{_AUDIO_ROOT_PATH}{_MIDI_SOLO_CHOIR}{_MP3}',
         model='MTT_musicnn', topN=10)